### Step 1: Install necessary Libraries

In [3]:
!pip install transformers torch tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------------------------------ --- 1.6/1.7 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 5.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Step 2: Loading the Summarization Model

In [4]:
# Loading the summarization model
from transformers import pipeline

# Load the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

### Step 3: Preprocessing the Subtitle File

In [5]:
import re

In [6]:
def preprocess_subtitles(file_path):
    # Open and read the subtitle file
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Join lines and remove timestamp patterns
    text = " ".join(lines)
    text = re.sub(r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}', '', text)  # Remove timestamps
    text = re.sub(r'\d+\n', '', text)  # Remove line numbers
    text = re.sub(r'\n+', ' ', text)  # Replace newlines with spaces

    # Return cleaned text
    return text.strip()

In [7]:
# Example usage
file_path = 'avatar_twow.srt'
cleaned_text = preprocess_subtitles(file_path)

In [8]:
cleaned_text

'ï»¿   (ENCHANTING MUSIC PLAYING)       (MUSIC FLOURISHES)       (SQUAWKING)       -(INSECTS AND BIRDS CHIRPING)  -(ANIMALS CALLING IN DISTANCE)       JAKE: <i>The forests of Pandora</i>       <i>hold many dangers.</i>       (SUSPENSEFUL MUSIC PLAYING)       (BREATHING HEAVILY)       <i>But the most dangerous thing</i>       <i>about Pandora...</i>       <i>is that you may grow  to love her too much.</i>       (GRUNTS)       (SINGING IN NA\'VI)       <i>We sing the songcords  to remember.</i>       <i>Each bead,</i>       <i>a story  in our life.</i>       <i>A bead for the birth  of our son.</i>       (GASPS IN AWE)       -Neteyam!  -(BABY SQUEALS)       NA\'VI PEOPLE: Neteyam!       (CONTINUES SINGING IN NA\'VI)       JAKE: <i>A bead</i>       <i>when we adopted  our daughter, Kiri.</i>       <i>Born of Grace\'s avatar.</i>       <i>A daughter whose conception  was a total mystery.</i>       (BABIES GURGLING)       (SIGHS IN AWE)       (NEYTIRI CONTINUES  SINGING IN NA\'VI)       <i>